In [1]:
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype
import sklearn.model_selection
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as sm

C:\Users\viet ha\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [17]:
df = pd.read_csv('facemask.csv')

In [2]:
def transform_res_9(data):
    dictz = {
        'Slightly Agree': 3,
        'Definitely Agree': 2,
        'Definitely Disagree': 1,
        'Slightly Disagree': 0
    }
    for c in data.columns[-25:-15]:
        data[c] = data[c].replace(dictz)
    return data

In [3]:
def transform_res_6(data):
    dictz = {
        'Disagree strongly': -2,
        'Disagree a little': -1,
        'Neutral; no opinion': 0,
        'Agree a little': 1,
        'Agree strongly': 2
    }
    for c in data.columns[-15:]:
        data[c] = data[c].replace(dictz)
    return data

In [4]:
# drop all responses' columns, add 2 summary variables
def add_summary_responses(data):
    data['res_6'] = data[data.columns[-15:]].sum(axis=1)
    data['res_9'] = data[data.columns[-25:-10]].sum(axis=1)
    data.drop(columns=list(data.columns[-27:-2]), inplace=True)
    return data

In [5]:
# fix the 'country' column
def fix_country(data):
    data.loc[data['country_text'] == 'Bulgaria', 'country'] = 'Bulgaria'
    data.loc[data['country_text'] == 'Poland', 'country'] = 'Poland'
    data.loc[data['country_text'] == 'Greece', 'country'] = 'Greece'
    data.loc[data['country_text'] == 'Ukraine', 'country'] = 'Ukraine'
    data.loc[data['country_text'] == 'ukraine', 'country'] = 'Ukraine'
    data.loc[data['country_text'] == 'Mexico', 'country'] = 'Mexico'
    data.loc[data['country_text'] == 'Portugal', 'country'] = 'Portugal'
    data.loc[data['country_text'] == 'sweden', 'country'] = 'Sweden'
    data.loc[data['country_text'] == '1', 'country'] = 'NaN'
    data.loc[data['country_text'] == 'The Netherlands', 'country'] = 'Netherlands'
    data.loc[data['country_text'] == 'the Netherlands', 'country'] = 'Netherlands'
    data.loc[data['country_text'] == 'Netherlands', 'country'] = 'Netherlands'
    data.loc[data['country_text'] == 'Belarus', 'country'] = 'Belarus'
    data.loc[data['country_text'] == 'The Russian Federation', 'country'] = 'Russia'
    data.loc[data['country_text'] == 'Russia', 'country'] = 'Russia'
    data.loc[data['country_text'] == 'first 3 years of my life: Australia; then: Germany', 'country'] = 'Australia, Germany'
    data.loc[data['country_text'] == 'Spain', 'country'] = 'Spain'
    data.loc[data['country_text'] == 'spain', 'country'] = 'Spain'
    data.loc[data['country_text'] == 'Armenia', 'country'] = 'Armenia'
    data.loc[data['country_text'] == 'China', 'country'] = 'China'
    data.loc[data['country_text'] == 'PRC', 'country'] = 'China'
    data.loc[data['country_text'] == 'Thailand (until the age of 11), England (from 12-21)', 'country'] = 'England, Thailand'
    data.loc[data['country_text'] == 'Estonia', 'country'] = 'Estonia'
    data.loc[data['country_text'] == 'HONG KONG', 'country'] = 'Hong Kong'
    data.loc[data['country_text'] == 'Hong Kong', 'country'] = 'Hong Kong'
    data.loc[data['country_text'] == 'HK', 'country'] = 'Hong Kong'
    data.loc[data['country_text'] == 'Italy', 'country'] = 'Italy'
    data.loc[data['country_text'] == 'Brazil', 'country'] = 'Brazil'
    data.loc[data['country_text'] == 'COLOMBIA', 'country'] = 'Colombia'
    data.loc[data['country_text'] == 'colombia', 'country'] = 'Colombia'
    data.loc[data['country_text'] == 'France', 'country'] = 'France'
    data.loc[data['country_text'] == 'Finland', 'country'] = 'Finland'
    data.loc[data['country_text'] == 'Belgium', 'country'] = 'Belgium'
    data.loc[data['country_text'] == 'Korea/US', 'country'] = 'Korea, United States'
    data.loc[data['country_text'] == 'Turkey', 'country'] = 'Turkey'
    data.loc[data['country_text'] == 'Kasachstan', 'country'] = 'Kazakhstan'
    data.loc[data['country_text'] == 'Romania', 'country'] = 'Romania'
    data.loc[data['country_text'] == 'Lithuania', 'country'] = 'Lithuania'
    data.loc[data['country_text'] == 'Taiwan', 'country'] = 'Taiwan'
    data.loc[data['country_text'] == 'Saudi Arabia', 'country'] = 'Saudi Arabia'
    return data

In [6]:
def transform_country(data, en_speak_country):
    for i in range(len(data)):
        for c in en_speak_country:
            if c in data['country'][i]:
                data.at[i, 'country'] = 'Yes'
    data.loc[data['country'] != 'Yes', 'country'] = 'No'
    return data

In [7]:
def drop_missings_unnecessaries(data):
    # now, we can drop 'country_text' without losing information 
    data.drop(['country_text'], axis=1, inplace=True)
    # data guide: fricative is just like target. You can ignore it. 
    # You can also ignore minimal_pair for the scope of this work.
    data.drop(['fricative', 'minimal_pair'], axis=1, inplace=True)
    # data guide: spreadsheet_name, spreadsheet_row and trial_number are 
    # most likely things you will not be touching during your analysis
    data.drop(['spreadsheet_name', 'spreadsheet_row', 'trial_number'], axis=1, inplace=True)
    return data

In [8]:
def transform_vaccine(data, predictor):
    cats = ['not sure', 'no', 'yes']
    cat_type = CategoricalDtype(categories=cats)
    data[predictor] = data[predictor].astype(cat_type)
    data = pd.get_dummies(data, prefix= predictor, columns=[predictor], drop_first=True)
    return data

In [9]:
def transform_gender(data, predictor):
    cats = ['prefer not to say', 'non-binary', 'no', 'yes']
    cat_type = CategoricalDtype(categories=cats)
    data[predictor] = data[predictor].astype(cat_type)
    data = pd.get_dummies(data, prefix= predictor, columns=[predictor], drop_first=False)
    return data

In [10]:
def transform_political(data, predictor):
    cats = ['prefer not to say', 'left/liberal', 'center', 'right/concervative']
    cat_type = CategoricalDtype(categories=cats)
    data[predictor] = data[predictor].astype(cat_type)
    data = pd.get_dummies(data, prefix= predictor, columns=[predictor], drop_first=False)
    return data

In [11]:
def transform_education(data, predictor):
    cats = ['Other (please specify)', 'high school', 'some college', 'BA/BSc', 'MA/MSc', 'PhD']
    cat_type = CategoricalDtype(categories=cats)
    data[predictor] = data[predictor].astype(cat_type)
    data = pd.get_dummies(data, prefix= predictor, columns=[predictor], drop_first=False)
    return data

In [12]:
def transform_target(data, predictor):
    cats = ['S', 'SH', 'F', 'TH', 'X']
    cat_type = CategoricalDtype(categories=cats)
    data[predictor] = data[predictor].astype(cat_type)
    data = pd.get_dummies(data, prefix= predictor, columns=[predictor], drop_first=False)
    # drop answer, because 'target' and 'correct' together make answer redundant
    data.drop(['answer'], axis=1, inplace=True)
    return data

In [13]:
def transform_binary(data, predictor):
    cats = ['No', 'Yes']
    cat_type = CategoricalDtype(categories=cats)
    data[predictor] = data[predictor].astype(cat_type)
    data = pd.get_dummies(data, prefix= predictor, columns=[predictor], drop_first=False)
    return data

In [14]:
def transform_condition(data, predictor):
    cats = ['NW', 'WM']
    cat_type = CategoricalDtype(categories=cats)
    data[predictor] = data[predictor].astype(cat_type)
    data = pd.get_dummies(data, prefix= predictor, columns=[predictor], drop_first=False)
    return data

In [15]:
def transform_logfreq(data):
    lf = data['logfreq'].unique()
    data['logfreq'] = data['logfreq'].replace(dict(zip(sorted(lf), np.arange(len(lf)))))
    # drop item, because item is now represented by its rank
    data.drop(['item'], axis=1, inplace=True)
    return data

In [16]:
def transform_syllable(data, predictor):
    cats = ['onset', 'coda']
    cat_type = CategoricalDtype(categories=cats)
    data[predictor] = data[predictor].astype(cat_type)
    data = pd.get_dummies(data, prefix= predictor, columns=[predictor], drop_first=False)
    return data

In [17]:
def transform_device(data, predictor):
    cats = ['mobile', 'computer', 'tablet']
    cat_type = CategoricalDtype(categories=cats)
    data[predictor] = data[predictor].astype(cat_type)
    data = pd.get_dummies(data, prefix= predictor, columns=[predictor], drop_first=False)
    return data

In [18]:
def transform_browser(data, predictor):
    for i in range(len(data)):
        for br in ['Safari', 'Firefox', 'Chrome', 'Edge', 'Opera']:
            if br in data['participant_browser'][i]:
                data.at[i, 'participant_browser'] = br
    cats = ['Safari', 'Firefox', 'Chrome', 'Edge', 'Opera']
    cat_type = CategoricalDtype(categories=cats)
    data[predictor] = data[predictor].astype(cat_type)
    data = pd.get_dummies(data, prefix= predictor, columns=[predictor], drop_first=False)
    return data

In [19]:
def transform_boolean(data, predictor):
    cats = [True, False]
    cat_type = CategoricalDtype(categories=cats)
    data[predictor] = data[predictor].astype(cat_type)
    data = pd.get_dummies(data, prefix= predictor, columns=[predictor], drop_first=False)
    return data

In [20]:
def standardize(data, c):
    data[c] = (data[c] - data[c].mean())/data[c].std()
    return data

In [21]:
def process_data(data):
    data = (
        data
        .pipe(fix_country)
        .pipe(transform_res_9)
        .pipe(transform_res_6)
        .pipe(add_summary_responses)
        .pipe(drop_missings_unnecessaries)
        .pipe(transform_vaccine, 'vaccine')
        .pipe(transform_gender, 'gender')
        .pipe(transform_political, 'political')
        .pipe(transform_education, 'education')
        .pipe(transform_binary, 'glasses')
        .pipe(transform_binary, 'impairment')
        .pipe(transform_condition, 'condition')
        .pipe(transform_logfreq)
        .pipe(transform_target, 'target')
        .pipe(transform_syllable, 'syllable')
        .pipe(transform_device, 'participant_device_type')
        .pipe(transform_browser, 'participant_browser')
        .pipe(transform_country, ['England', 'United States', 'Australia'])
        .pipe(transform_binary, 'country')
        #.pipe(transform_boolean, 'correct')
        .pipe(transform_boolean, 'visual_cues')
        .pipe(standardize, 'mask_before')
        .pipe(standardize, 'mask_people')
        .pipe(standardize, 'mask_perception')
        .pipe(standardize, 'mask_spread')
        .pipe(standardize, 'mask_freedom')
        .pipe(standardize, 'mask_vulnerable')
        .pipe(standardize, 'intensity')
        .pipe(standardize, 'cog')
        .pipe(standardize, 'f1')
        .pipe(standardize, 'f2')
        .pipe(standardize, 'logfreq')
        .pipe(standardize, 'rt')
    ) 
    
    # for now, just remove country, vowel, and don't care about who took the experiment
    X = data.drop(['correct', 'vowel', 'participant_private_id'], axis = 1)
    y = data['correct']
    y = [x+0 for x in y]
    data = data.drop(['vowel', 'participant_private_id'], axis = 1)
    data['correct'] = y
    return X, y, data

In [22]:
df = pd.read_csv('facemask.csv')
#df.pipe(fix_country)
X, y, data = process_data(df)
data

,rt,correct,logfreq,intensity,cog,f1,f2,loudness_accuracy,mgcurk_rate,age,...,participant_device_type_tablet,participant_browser_Safari,participant_browser_Firefox,participant_browser_Chrome,participant_browser_Edge,participant_browser_Opera,country_No,country_Yes,visual_cues_True,visual_cues_False
0,-0.114404,1,-0.357448,1.827673,0.598759,0.389142,-0.000216,0.8,0.5,19,...,0,1,0,0,0,0,1,0,0,1
1,-0.114404,1,-0.357448,0.752391,0.945420,0.772389,0.294794,0.8,0.5,19,...,0,1,0,0,0,0,1,0,0,1
2,-0.072996,1,1.281805,0.731831,0.309690,1.742549,0.625340,0.8,0.5,19,...,0,1,0,0,0,0,1,0,0,1
3,-0.072996,1,1.281805,-0.117329,0.342363,0.986356,0.686283,0.8,0.5,19,...,0,1,0,0,0,0,1,0,0,1
4,-0.128597,0,0.917526,-0.536751,-1.076401,-0.576647,1.372278,0.8,0.5,19,...,0,1,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27195,-0.109228,1,1.555014,-1.371132,-1.327813,1.538468,0.702497,1.0,1.0,34,...,0,0,0,1,0,0,0,1,1,0
27196,0.019672,1,0.644318,1.432830,-0.706481,1.805643,0.691699,1.0,1.0,34,...,0,0,0,1,0,0,0,1,0,1
27197,0.019672,1,0.644318,0.463077,-1.672813,0.678121,0.317890,1.0,1.0,34,...,0,0,0,1,0,0,0,1,0,1
27198,-0.097373,1,0.735387,-0.667149,-0.944049,-1.349019,-0.257677,1.0,1.0,34,...,0,0,0,1,0,0,0,1,1,0


In [120]:
def train_test_split(df, n_samples=df.shape[0], validation=False):
    if validation:
        sample = df.sample(n=n_samples)

        msk = np.random.rand(len(sample)) < 0.8
        non_test = sample[msk]
        test = sample[~msk]
        
        msk = np.random.rand(len(non_test)) < 0.7
        train = non_test[msk]
        validation = non_test[~msk]
        
        return train, validation, test
    
    else:
        sample = df.sample(n=n_samples)

        msk = np.random.rand(len(sample)) < 0.8
        train = sample[msk]
        test = sample[~msk]
        
        return train, test

In [121]:
train, validation, test = train_test_split(data, validation=True)

y_train = train['correct'].values
y_val = validation['correct'].values
y_test = test['correct'].values

In [122]:
y_train = [y+0 for y in y_train]
y_test = [y+0 for y in y_test]

In [123]:
all_predictors = list(X.columns)

predictors = [([], 0)]

regression_model = LinearRegression(fit_intercept=True)

R_sq_fwd = []

for k in range(1, len(all_predictors)):
    best_k_minus_1 = predictors[-1][0]

    new_predictors = list(set(all_predictors) - set(best_k_minus_1))
    validation_R_sqs = []

    for predictor in new_predictors:

        k_predictors = best_k_minus_1 + [predictor]
        
        X_train = train[k_predictors].values
        X_val = validation[k_predictors].values
        
        if k == 1:
            X_train = X_train.reshape((len(X_train), 1))
            
        regression_model.fit(X_train, y_train)
        validation_R_sqs.append(regression_model.score(X_val, y_val))
    
    best_k = best_k_minus_1 + [new_predictors[np.argmax(validation_R_sqs)]]
    R_sq_fwd.append(np.max(validation_R_sqs))
    predictors.append((best_k, np.max(validation_R_sqs)))

X_train = train[all_predictors].values
X_val = validation[all_predictors].values  
regression_model.fit(X_train, y_train)

predictors.append((all_predictors, regression_model.score(X_val, y_val)))

In [124]:
best_predictor_set = sorted(predictors, key=lambda t: t[1])[-1]

X_train = train[best_predictor_set[0]].values
X_val = validation[best_predictor_set[0]].values  
X_test = test[best_predictor_set[0]].values  

regression_model.fit(np.vstack((X_train, X_val)), np.hstack((y_train, y_val)))

print('best predictor set: {}\nvalidation R^2: {}\ntest R^2: {}'.format(best_predictor_set[0], best_predictor_set[1], regression_model.score(X_test, y_test)))

best predictor set: ['target_X', 'target_TH', 'target_F', 'condition_WM', 'logfreq', 'f2', 'intensity', 'syllable_coda', 'f1', 'cog', 'mask_spread', 'country_Yes', 'impairment_Yes', 'syllable_onset', 'vaccine_yes', 'participant_browser_Edge', 'gender_non-binary', 'rt', 'education_some college', 'glasses_Yes', 'participant_browser_Safari', 'participant_browser_Opera', 'education_high school', 'visual_cues_False', 'political_center']
validation R^2: 0.3061080620097769
test R^2: 0.3239833013454233


In [129]:
rt_md = sm.ols(formula='correct ~ target_X + target_TH + target_F + condition_WM + logfreq + intensity + syllable_coda + f1 + f2 + cog + country_Yes + impairment_Yes + vaccine_yes + mask_spread + syllable_onset + rt + participant_browser_Safari + participant_browser_Edge + participant_browser_Opera + rt', data=data).fit()
rt_md.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                correct   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.303
Method:                 Least Squares   F-statistic:                     658.7
Date:                Sun, 20 Mar 2022   Prob (F-statistic):               0.00
Time:                        01:32:53   Log-Likelihood:                -7233.3
No. Observations:               27200   AIC:                         1.450e+04
Df Residuals:                   27181   BIC:                         1.466e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                  -6.337e+11   3.94e+11     -1.608      0.108   -1.41e+12    1.39e+11
target_X                      -0.9954      0.013    -76.378      0.000      -1.021      -0.970
target_TH                     -0.3615      0.008    -44.322      0.000      -0.377      -0.346
target_F                      -0.1906      0.009    -21.690      0.000      -0.208      -0.173
condition_WM                  -0.0603      0.004    -13.562      0.000      -0.069      -0.052
logfreq                        0.0309      0.002     15.626      0.000       0.027       0.035
intensity                     -0.0530      0.004    -12.569      0.000      -0.061      -0.045
syllable_coda               6.337e+11   3.94e+11      1.608      0.108   -1.39e+11    1.41e+12
f1                            -0.0228      0.002    -10.965      0.000      -0.027      -0.019
f2                            -0.0225      0.002    -10.808      0.000      -0.027      -0.018
cog                           -0.0163      0.002     -7.294      0.000      -0.021      -0.012
country_Yes                    0.0359      0.005      7.080      0.000       0.026       0.046
impairment_Yes                -0.0554      0.011     -4.821      0.000      -0.078      -0.033
vaccine_yes                    0.0374      0.006      5.851      0.000       0.025       0.050
mask_spread                   -0.0139      0.002     -7.015      0.000      -0.018      -0.010
syllable_onset              6.337e+11   3.94e+11      1.608      0.108   -1.39e+11    1.41e+12
rt                            -0.0044      0.002     -2.295      0.022      -0.008      -0.001
participant_browser_Safari    -0.0216      0.011     -2.049      0.040      -0.042      -0.001
participant_browser_Edge      -0.0120      0.009     -1.378      0.168      -0.029       0.005
participant_browser_Opera      0.0199      0.015      1.359      0.174      -0.009       0.049
==============================================================================
Omnibus:                     7184.586   Durbin-Watson:                   1.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            15004.042
Skew:                          -1.587   Prob(JB):                         0.00
Kurtosis:                       4.780   Cond. No.                     5.94e+14
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.14e-25. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""